In [1]:
import pandas as pd
import os
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop", "Data_cleaning_cosine_calculation_semantic_and_analysis")
os.chdir(desktop_path)

In [2]:
critical_reasoning_data = pd.read_csv("NLP_analysis/critical_reasoning_analysis.csv")
critical_reasoning_data

,QuestionID,deepseek-r1:1.5b,deepseek-r1:14b,qwen2.5:1.5b,qwen2.5:14b,deepseek-r1:1.5b_time,deepseek-r1:14b_time,qwen2.5:1.5b_time,qwen2.5:14b_time,Answer,...,qwen2.5:1.5b_reasoning,qwen2.5:1.5b_solution,qwen2.5:1.5b_structure_score,qwen2.5:14b_reasoning,qwen2.5:14b_solution,qwen2.5:14b_structure_score,deepseek-r1:1.5b_correct,deepseek-r1:14b_correct,qwen2.5:1.5b_correct,qwen2.5:14b_correct
0,1,# Argument Construction\n\nSituation: The stim...,# Argument Construction\n\nSituation: The argu...,# Argument Construction\n\nSituation: The grow...,# Argument Construction\n\nSituation: The argu...,8.6,167.4,9.0,125.3,E,...,Situation: The growing popularity of computer-...,D,0,Situation: The argument posits that despite ex...,E,0,0,1,0,1
1,3,# Argument Construction\n\nSituation: The stim...,# Argument Construction\n\nSituation: The argu...,# Argument Construction\n\nSituation: Homeowne...,# Argument Construction\n\nSituation: The argu...,5.4,91.1,3.1,90.7,A,...,Situation: Homeowners aged 40 to 50 are statis...,C,0,Situation: The argument claims that homeowners...,A,0,0,1,0,1
2,4,# Argument Construction\n\nSituation: \nSunco...,"# Argument Construction\n\nSituation: Suncorp,...","# Argument Construction\n\nSituation: Suncorp,...","# Argument Construction\n\nSituation: Suncorp,...",8.8,97.2,3.5,87.6,E,...,"Situation: Suncorp, a new corporation with lim...",D,0,"Situation: Suncorp, despite being a corporatio...",E,0,1,1,0,1
3,5,# Argument Construction\n\nSituation: The pred...,# Argument Construction\n\nSituation: A 1940 p...,"# Argument Construction\n\nSituation: In 1940,...",# Argument Construction\n\nSituation: The pred...,5.9,109.2,2.8,94.7,D,...,"Situation: In 1940, a prediction was made that...",C,0,Situation: The prediction made in 1940 posits ...,D,0,0,1,0,1
4,6,# Argument Construction\n\nSituation: The comp...,# Argument Construction\n\nSituation: A compan...,# Argument Construction\n\nSituation: The comp...,# Argument Construction\n\nSituation: The comp...,6.6,105.0,4.2,90.7,D,...,Situation: The company is considering changing...,A,0,Situation: The company is considering altering...,D,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,328,# Argument Construction\n\nSituation: The argu...,# Argument Construction\n\nSituation: The cons...,# Argument Construction\n\nSituation: The cons...,# Argument Construction\n\nSituation: A consum...,7.3,121.7,4.1,90.9,A,...,Situation: The consumer health advocate critic...,D,0,Situation: A consumer health advocate argues t...,D,0,1,1,0,0
327,329,# Argument Construction\n\nSituation: XYZ Comp...,# Argument Construction\n\nSituation: XYZ Comp...,# Argument Construction\n\nSituation: XYZ Comp...,# Argument Construction\n\nSituation: XYZ Comp...,6.7,115.4,3.8,86.4,A,...,Situation: XYZ Company asked volunteers from i...,E,0,Situation: XYZ Company is considering a plan t...,A,0,1,0,0,1
328,330,# Argument Construction\n\nSituation: The argu...,# Argument Construction\n\nSituation: The poli...,# Argument Construction\n\nSituation: In the c...,# Argument Construction\n\nSituation: A politi...,7.0,124.7,4.2,88.6,C,...,Situation: In the context of a political adver...,B,0,Situation: A political advertisement presents ...,C,0,0,0,0,1
329,331,# Argument Construction\n\nSituation: The argu...,# Argument Construction\n\nSituation: Vitacorp...,# Argument Construction\n\nSituation: Vitacorp...,# Argument Construction\n\nSituation: \nVitaco...,4.6,106.4,3.5,92.4,C,...,"Situation: Vitacorp, a manufacturer, wishes to...",D,0,Situation: \nVitacorp aims to increase the eff...,C,0,1,1,0,1


In [3]:
corpus = critical_reasoning_data["Solution"].tolist() + \
         critical_reasoning_data["deepseek-r1:1.5b_reasoning"].tolist() + \
         critical_reasoning_data["deepseek-r1:14b_reasoning"].tolist() + \
         critical_reasoning_data["qwen2.5:1.5b_reasoning"].tolist() + \
         critical_reasoning_data["qwen2.5:14b_reasoning"].tolist()

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
## Analyze only words, min_df <- words appears at least in 1% of the documents,
## max_df <- words appears in at most 75% of the documents, sublinear_tf <- use sublinear term frequency scaling 1 + log(tf)
vectorizer = TfidfVectorizer(analyzer='word', min_df=0.01, max_df=0.75, ngram_range=(1, 2), stop_words='english', sublinear_tf=True)
X = vectorizer.fit_transform(corpus)

In [5]:
n = len(critical_reasoning_data)
Solution = X[0:n]
deepseek_r1_1_5b = X[n:2*n]
deepseek_r1_14b = X[2*n:3*n]
qwen2_5_1_5b = X[3*n:4*n]
qwen2_5_14b = X[4*n:5*n]

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

deepseek_r1_1_5b_similarity = cosine_similarity(Solution, deepseek_r1_1_5b)
deepseek_r1_14b_similarity = cosine_similarity(Solution, deepseek_r1_14b)   
qwen2_5_1_5b_similarity = cosine_similarity(Solution, qwen2_5_1_5b)
qwen2_5_14b_similarity = cosine_similarity(Solution, qwen2_5_14b)

In [7]:
## The result is a matrix where the cell (i,j) contains the cosine similarity between the i-th solution and the j-th reasoning.
## But I want only the cosine similarity between the i-th solution and the i-th reasoning.
deepseek_r1_1_5b_similarity = deepseek_r1_1_5b_similarity.diagonal()
deepseek_r1_14b_similarity = deepseek_r1_14b_similarity.diagonal()
qwen2_5_1_5b_similarity = qwen2_5_1_5b_similarity.diagonal()
qwen2_5_14b_similarity = qwen2_5_14b_similarity.diagonal()

In [8]:
## export the vectors to CSV files
import os

output_dir = "vector_exports"
os.makedirs(output_dir, exist_ok=True)

def export_vector(vector, filename):
    filepath = os.path.join(output_dir, filename)
    pd.DataFrame(vector).to_csv(filepath, index=False, header=False)

export_vector(deepseek_r1_1_5b_similarity, "deepseek_r1_1_5b_similarity_crit.csv")
export_vector(deepseek_r1_14b_similarity, "deepseek_r1_14b_similarity_crit.csv")
export_vector(qwen2_5_1_5b_similarity, "qwen2_5_1_5b_similarity_crit.csv")
export_vector(qwen2_5_14b_similarity, "qwen2_5_14b_similarity_crit.csv")

In [9]:
# Calculate the mean cosine similarity for each model, separating questions with correct and incorrect answers

def mean_similarity_by_correctness(similarity, model_solution_col):
    correct_mask = critical_reasoning_data[model_solution_col] == critical_reasoning_data['Answer']
    mean_correct = similarity[correct_mask.values].mean()
    mean_wrong = similarity[~correct_mask.values].mean()
    return mean_correct, mean_wrong

mean_corr_dk1, mean_wrong_dk1 = mean_similarity_by_correctness(deepseek_r1_1_5b_similarity, 'deepseek-r1:1.5b_reasoning')
mean_corr_dk14, mean_wrong_dk14 = mean_similarity_by_correctness(deepseek_r1_14b_similarity, 'deepseek-r1:14b_reasoning')
mean_corr_qw1, mean_wrong_qw1 = mean_similarity_by_correctness(qwen2_5_1_5b_similarity, 'qwen2.5:1.5b_reasoning')
mean_corr_qw14, mean_wrong_qw14 = mean_similarity_by_correctness(qwen2_5_14b_similarity, 'qwen2.5:14b_reasoning')

print("DeepSeek R1 1.5B - Mean similarity (correct):", mean_corr_dk1)
print("DeepSeek R1 1.5B - Mean similarity (wrong):", mean_wrong_dk1)
print("DeepSeek R1 14B - Mean similarity (correct):", mean_corr_dk14)
print("DeepSeek R1 14B - Mean similarity (wrong):", mean_wrong_dk14)
print("Qwen2.5 1.5B - Mean similarity (correct):", mean_corr_qw1)
print("Qwen2.5 1.5B - Mean similarity (wrong):", mean_wrong_qw1)
print("Qwen2.5 14B - Mean similarity (correct):", mean_corr_qw14)
print("Qwen2.5 14B - Mean similarity (wrong):", mean_wrong_qw14)

DeepSeek R1 1.5B - Mean similarity (correct): nan
DeepSeek R1 1.5B - Mean similarity (wrong): 0.3313429028235462
DeepSeek R1 14B - Mean similarity (correct): nan
DeepSeek R1 14B - Mean similarity (wrong): 0.3887031518416355
Qwen2.5 1.5B - Mean similarity (correct): nan
Qwen2.5 1.5B - Mean similarity (wrong): 0.42328633059681386
Qwen2.5 14B - Mean similarity (correct): nan
Qwen2.5 14B - Mean similarity (wrong): 0.4532586456791295


C:\Users\filip\AppData\Local\Temp\ipykernel_17756\876451440.py:5: RuntimeWarning: Mean of empty slice.
  mean_correct = similarity[correct_mask.values].mean()
c:\Users\filip\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [10]:
mean_deepseek_r1_1_5b_similarity = deepseek_r1_1_5b_similarity.mean()
mean_deepseek_r1_14b_similarity = deepseek_r1_14b_similarity.mean()
mean_qwen2_5_1_5b_similarity = qwen2_5_1_5b_similarity.mean()
mean_qwen2_5_14b_similarity = qwen2_5_14b_similarity.mean()

In [11]:
print("DeepSeek R1 1.5B Similarity:")
print(deepseek_r1_1_5b_similarity.mean(axis=0))
print("DeepSeek R1 14B Similarity:")
print(deepseek_r1_14b_similarity.mean(axis=0))
print("Qwen2.5 1.5B Similarity:")
print(qwen2_5_1_5b_similarity.mean(axis=0))
print("Qwen2.5 14B Similarity:")
print(qwen2_5_14b_similarity.mean(axis=0))

DeepSeek R1 1.5B Similarity:
0.3313429028235462
DeepSeek R1 14B Similarity:
0.3887031518416355
Qwen2.5 1.5B Similarity:
0.42328633059681386
Qwen2.5 14B Similarity:
0.4532586456791295


In [12]:
import numpy as np


stats = {

    "Model": [
        "DeepSeek R1 1.5B",
        "DeepSeek R1 14B",
        "Qwen2.5 1.5B",
        "Qwen2.5 14B"
    ],
    "Mean": [
        np.mean(deepseek_r1_1_5b_similarity),
        np.mean(deepseek_r1_14b_similarity),
        np.mean(qwen2_5_1_5b_similarity),
        np.mean(qwen2_5_14b_similarity)
    ],
    "Median": [
        np.median(deepseek_r1_1_5b_similarity),
        np.median(deepseek_r1_14b_similarity),
        np.median(qwen2_5_1_5b_similarity),
        np.median(qwen2_5_14b_similarity)
    ],
    "Std": [
        np.std(deepseek_r1_1_5b_similarity),
        np.std(deepseek_r1_14b_similarity),
        np.std(qwen2_5_1_5b_similarity),
        np.std(qwen2_5_14b_similarity)
    ]
}

stats_df = pd.DataFrame(stats)
print(stats_df)

              Model      Mean    Median       Std
0  DeepSeek R1 1.5B  0.331343  0.333656  0.111548
1   DeepSeek R1 14B  0.388703  0.394972  0.103016
2      Qwen2.5 1.5B  0.423286  0.431762  0.106928
3       Qwen2.5 14B  0.453259  0.459645  0.098620


N-GRAMS OVERLAPPING

In [13]:
### BUILD FUNCTION FOR N-GRAM OVERLAP ###

def ngrams(text, n):
    tokens = text.lower().split() 
    return set([' '.join(tokens[i:i+n]) for i in range(len(tokens)-n+1)])

def ngram_overlap(text1, text2, n):
    ngrams1 = ngrams(text1, n)
    ngrams2 = ngrams(text2, n)
    
    intersection = len(ngrams1 & ngrams2)
    union = len(ngrams1 | ngrams2)
    
    if union == 0:
        return 0.0
    else:
        return intersection / union

In [14]:
# N-gram overlap with n=3 (can change with n=2, n=1)
n = 3

critical_reasoning_data['deepseek_r1_1_5b_ngram_overlap'] = critical_reasoning_data.apply(lambda row: ngram_overlap(row['Solution'], row['deepseek-r1:1.5b_reasoning'], n), axis=1)
critical_reasoning_data['deepseek_r1_14b_ngram_overlap']  = critical_reasoning_data.apply(lambda row: ngram_overlap(row['Solution'], row['deepseek-r1:14b_reasoning'], n), axis=1)
critical_reasoning_data['qwen2_5_1_5b_ngram_overlap']     = critical_reasoning_data.apply(lambda row: ngram_overlap(row['Solution'], row['qwen2.5:1.5b_reasoning'], n), axis=1)
critical_reasoning_data['qwen2_5_14b_ngram_overlap']      = critical_reasoning_data.apply(lambda row: ngram_overlap(row['Solution'], row['qwen2.5:14b_reasoning'], n), axis=1)

In [15]:
# DeepSeek R1 1.5B
mean_corr_dk1, mean_wrong_dk1 = mean_similarity_by_correctness(critical_reasoning_data['deepseek_r1_1_5b_ngram_overlap'], 'deepseek-r1:1.5b_solution')
print("DeepSeek R1 1.5B - Mean overlap (correct):", mean_corr_dk1)
print("DeepSeek R1 1.5B - Mean overlap (wrong):", mean_wrong_dk1)

# DeepSeek R1 14B
mean_corr_dk14, mean_wrong_dk14 = mean_similarity_by_correctness(critical_reasoning_data['deepseek_r1_14b_ngram_overlap'], 'deepseek-r1:14b_solution')
print("DeepSeek R1 14B - Mean overlap (correct):", mean_corr_dk14)
print("DeepSeek R1 14B - Mean overlap (wrong):", mean_wrong_dk14)

# Qwen2.5 1.5B
mean_corr_qw1, mean_wrong_qw1 = mean_similarity_by_correctness(critical_reasoning_data['qwen2_5_1_5b_ngram_overlap'], 'qwen2.5:1.5b_solution')
print("Qwen2.5 1.5B - Mean overlap (correct):", mean_corr_qw1)
print("Qwen2.5 1.5B - Mean overlap (wrong):", mean_wrong_qw1)

# Qwen2.5 14B
mean_corr_qw14, mean_wrong_qw14 = mean_similarity_by_correctness(critical_reasoning_data['qwen2_5_14b_ngram_overlap'], 'qwen2.5:14b_solution')
print("Qwen2.5 14B - Mean overlap (correct):", mean_corr_qw14)
print("Qwen2.5 14B - Mean overlap (wrong):", mean_wrong_qw14)


DeepSeek R1 1.5B - Mean overlap (correct): 0.008288042571614475
DeepSeek R1 1.5B - Mean overlap (wrong): 0.00754410778690675
DeepSeek R1 14B - Mean overlap (correct): 0.012010699695175682
DeepSeek R1 14B - Mean overlap (wrong): 0.011785170944997017
Qwen2.5 1.5B - Mean overlap (correct): 0.018182528069243697
Qwen2.5 1.5B - Mean overlap (wrong): 0.019041218400533563
Qwen2.5 14B - Mean overlap (correct): 0.023938988646879614
Qwen2.5 14B - Mean overlap (wrong): 0.02110025145781824


In [16]:
mean_deepseek_r1_1_5b_ngram = critical_reasoning_data['deepseek_r1_1_5b_ngram_overlap'].mean()
mean_deepseek_r1_14b_ngram  = critical_reasoning_data['deepseek_r1_14b_ngram_overlap'].mean()
mean_qwen2_5_1_5b_ngram     = critical_reasoning_data['qwen2_5_1_5b_ngram_overlap'].mean()
mean_qwen2_5_14b_ngram      = critical_reasoning_data['qwen2_5_14b_ngram_overlap'].mean()

In [17]:
print("Mean N-gram overlap (DeepSeek 1.5B):", mean_deepseek_r1_1_5b_ngram)
print("Mean N-gram overlap (DeepSeek 14B):", mean_deepseek_r1_14b_ngram)
print("Mean N-gram overlap (Qwen 1.5B):", mean_qwen2_5_1_5b_ngram)
print("Mean N-gram overlap (Qwen 14B):", mean_qwen2_5_14b_ngram)

Mean N-gram overlap (DeepSeek 1.5B): 0.00773065336736216
Mean N-gram overlap (DeepSeek 14B): 0.011948696322770068
Mean N-gram overlap (Qwen 1.5B): 0.018781795339720915
Mean N-gram overlap (Qwen 14B): 0.02333007523170331


METRICHE / SCORE

In [18]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import single_meteor_score
from rouge_score import rouge_scorer
import sacrebleu

In [19]:
### BLEU SCORE ###

#smoothing function in order to prevent zero division errors
smoothie = SmoothingFunction().method4

def compute_bleu(reference_text, candidate_text):
    reference = [reference_text.lower().split()]
    candidate = candidate_text.lower().split()
    return sentence_bleu(reference, candidate, smoothing_function=smoothie)

In [20]:
### ROUGE-L ###

rouge_scorer_obj = rouge_scorer.RougeScorer(['rougeL'], use_stemmer = True)

def compute_rouge(reference_text, candidate_text):
    scores = rouge_scorer_obj.score(reference_text, candidate_text)
    #F1 measure of ROUGE-L
    return scores['rougeL'].fmeasure 

In [21]:
### METEOR SCORE ###

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

def compute_meteor(reference_text, candidate_text):
    return single_meteor_score(reference_text.lower().split(), candidate_text.lower().split())

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\filip\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\filip\AppData\Roaming\nltk_data...


In [22]:
# BLEU
critical_reasoning_data['deepseek_r1_1_5b_bleu'] = critical_reasoning_data.apply(lambda row: compute_bleu(row['Solution'], row['deepseek-r1:1.5b_reasoning']), axis=1)
critical_reasoning_data['deepseek_r1_14b_bleu']  = critical_reasoning_data.apply(lambda row: compute_bleu(row['Solution'], row['deepseek-r1:14b_reasoning']), axis=1)
critical_reasoning_data['qwen2_5_1_5b_bleu']     = critical_reasoning_data.apply(lambda row: compute_bleu(row['Solution'], row['qwen2.5:1.5b_reasoning']), axis=1)
critical_reasoning_data['qwen2_5_14b_bleu']      = critical_reasoning_data.apply(lambda row: compute_bleu(row['Solution'], row['qwen2.5:14b_reasoning']), axis=1)

In [23]:
# ROUGE-L
critical_reasoning_data['deepseek_r1_1_5b_rouge_l'] = critical_reasoning_data.apply(lambda row: compute_rouge(row['Solution'], row['deepseek-r1:1.5b_reasoning']), axis=1)
critical_reasoning_data['deepseek_r1_14b_rouge_l']  = critical_reasoning_data.apply(lambda row: compute_rouge(row['Solution'], row['deepseek-r1:14b_reasoning']), axis=1)   
critical_reasoning_data['qwen2_5_1_5b_rouge_l']     = critical_reasoning_data.apply(lambda row: compute_rouge(row['Solution'], row['qwen2.5:1.5b_reasoning']), axis=1)
critical_reasoning_data['qwen2_5_14b_rouge_l']      = critical_reasoning_data.apply(lambda row: compute_rouge(row['Solution'], row['qwen2.5:14b_reasoning']), axis=1)

In [24]:
# METOR
critical_reasoning_data['deepseek_r1_1_5b_meteor'] = critical_reasoning_data.apply(lambda row: compute_meteor(row['Solution'], row['deepseek-r1:1.5b_reasoning']), axis=1)
critical_reasoning_data['deepseek_r1_14b_meteor']  = critical_reasoning_data.apply(lambda row: compute_meteor(row['Solution'], row['deepseek-r1:14b_reasoning']), axis=1)
critical_reasoning_data['qwen2_5_1_5b_meteor']     = critical_reasoning_data.apply(lambda row: compute_meteor(row['Solution'], row['qwen2.5:1.5b_reasoning']), axis=1)
critical_reasoning_data['qwen2_5_14b_meteor']      = critical_reasoning_data.apply(lambda row: compute_meteor(row['Solution'], row['qwen2.5:14b_reasoning']), axis=1)

In [25]:
# Function to calculate mean metric by correctness

def mean_metric_by_correctness(metric_col, model_solution_col):
    correct_mask = critical_reasoning_data[model_solution_col] == critical_reasoning_data['Answer']
    mean_correct = critical_reasoning_data.loc[correct_mask, metric_col].mean()
    mean_wrong = critical_reasoning_data.loc[~correct_mask, metric_col].mean()
    return mean_correct, mean_wrong

# BLEU
mean_bleu_corr_dk1, mean_bleu_wrong_dk1 = mean_metric_by_correctness('deepseek_r1_1_5b_bleu', 'deepseek-r1:1.5b_solution')
mean_bleu_corr_dk14, mean_bleu_wrong_dk14 = mean_metric_by_correctness('deepseek_r1_14b_bleu', 'deepseek-r1:14b_solution')
mean_bleu_corr_qw1, mean_bleu_wrong_qw1 = mean_metric_by_correctness('qwen2_5_1_5b_bleu', 'qwen2.5:1.5b_solution')
mean_bleu_corr_qw14, mean_bleu_wrong_qw14 = mean_metric_by_correctness('qwen2_5_14b_bleu', 'qwen2.5:14b_solution')

print("BLEU - DeepSeek R1 1.5B (correct):", mean_bleu_corr_dk1)
print("BLEU - DeepSeek R1 1.5B (wrong):", mean_bleu_wrong_dk1)
print("BLEU - DeepSeek R1 14B (correct):", mean_bleu_corr_dk14)
print("BLEU - DeepSeek R1 14B (wrong):", mean_bleu_wrong_dk14)
print("BLEU - Qwen2.5 1.5B (correct):", mean_bleu_corr_qw1)
print("BLEU - Qwen2.5 1.5B (wrong):", mean_bleu_wrong_qw1)
print("BLEU - Qwen2.5 14B (correct):", mean_bleu_corr_qw14)
print("BLEU - Qwen2.5 14B (wrong):", mean_bleu_wrong_qw14)

# ROUGE-L
mean_rouge_corr_dk1, mean_rouge_wrong_dk1 = mean_metric_by_correctness('deepseek_r1_1_5b_rouge_l', 'deepseek-r1:1.5b_solution')
mean_rouge_corr_dk14, mean_rouge_wrong_dk14 = mean_metric_by_correctness('deepseek_r1_14b_rouge_l', 'deepseek-r1:14b_solution')
mean_rouge_corr_qw1, mean_rouge_wrong_qw1 = mean_metric_by_correctness('qwen2_5_1_5b_rouge_l', 'qwen2.5:1.5b_solution')
mean_rouge_corr_qw14, mean_rouge_wrong_qw14 = mean_metric_by_correctness('qwen2_5_14b_rouge_l', 'qwen2.5:14b_solution')

print("ROUGE-L - DeepSeek R1 1.5B (correct):", mean_rouge_corr_dk1)
print("ROUGE-L - DeepSeek R1 1.5B (wrong):", mean_rouge_wrong_dk1)
print("ROUGE-L - DeepSeek R1 14B (correct):", mean_rouge_corr_dk14)
print("ROUGE-L - DeepSeek R1 14B (wrong):", mean_rouge_wrong_dk14)
print("ROUGE-L - Qwen2.5 1.5B (correct):", mean_rouge_corr_qw1)
print("ROUGE-L - Qwen2.5 1.5B (wrong):", mean_rouge_wrong_qw1)
print("ROUGE-L - Qwen2.5 14B (correct):", mean_rouge_corr_qw14)
print("ROUGE-L - Qwen2.5 14B (wrong):", mean_rouge_wrong_qw14)

# METEOR
mean_meteor_corr_dk1, mean_meteor_wrong_dk1 = mean_metric_by_correctness('deepseek_r1_1_5b_meteor', 'deepseek-r1:1.5b_solution')
mean_meteor_corr_dk14, mean_meteor_wrong_dk14 = mean_metric_by_correctness('deepseek_r1_14b_meteor', 'deepseek-r1:14b_solution')
mean_meteor_corr_qw1, mean_meteor_wrong_qw1 = mean_metric_by_correctness('qwen2_5_1_5b_meteor', 'qwen2.5:1.5b_solution')
mean_meteor_corr_qw14, mean_meteor_wrong_qw14 = mean_metric_by_correctness('qwen2_5_14b_meteor', 'qwen2.5:14b_solution')

print("METEOR - DeepSeek R1 1.5B (correct):", mean_meteor_corr_dk1)
print("METEOR - DeepSeek R1 1.5B (wrong):", mean_meteor_wrong_dk1)
print("METEOR - DeepSeek R1 14B (correct):", mean_meteor_corr_dk14)
print("METEOR - DeepSeek R1 14B (wrong):", mean_meteor_wrong_dk14)
print("METEOR - Qwen2.5 1.5B (correct):", mean_meteor_corr_qw1)
print("METEOR - Qwen2.5 1.5B (wrong):", mean_meteor_wrong_qw1)
print("METEOR - Qwen2.5 14B (correct):", mean_meteor_corr_qw14)
print("METEOR - Qwen2.5 14B (wrong):", mean_meteor_wrong_qw14)

BLEU - DeepSeek R1 1.5B (correct): 0.01951380579630338
BLEU - DeepSeek R1 1.5B (wrong): 0.017820978203230916
BLEU - DeepSeek R1 14B (correct): 0.02776072865091069
BLEU - DeepSeek R1 14B (wrong): 0.027386172652331068
BLEU - Qwen2.5 1.5B (correct): 0.05816313770099355
BLEU - Qwen2.5 1.5B (wrong): 0.057900473144140374
BLEU - Qwen2.5 14B (correct): 0.07485536487392783
BLEU - Qwen2.5 14B (wrong): 0.06571832900393632
ROUGE-L - DeepSeek R1 1.5B (correct): 0.19310834411248243
ROUGE-L - DeepSeek R1 1.5B (wrong): 0.1834862916920238
ROUGE-L - DeepSeek R1 14B (correct): 0.2230839622414642
ROUGE-L - DeepSeek R1 14B (wrong): 0.21246254837714673
ROUGE-L - Qwen2.5 1.5B (correct): 0.23390798109854669
ROUGE-L - Qwen2.5 1.5B (wrong): 0.23275555939122228
ROUGE-L - Qwen2.5 14B (correct): 0.2542941480257051
ROUGE-L - Qwen2.5 14B (wrong): 0.23788535055747462
METEOR - DeepSeek R1 1.5B (correct): 0.1503686879660947
METEOR - DeepSeek R1 1.5B (wrong): 0.14095936727816744
METEOR - DeepSeek R1 14B (correct): 0.164

In [26]:
print("Mean BLEU DeepSeek_1.5:", critical_reasoning_data['deepseek_r1_1_5b_bleu'].mean())
print("Mean BLEU DeepSeek_14B:", critical_reasoning_data['deepseek_r1_14b_bleu'].mean())
print("Mean BLEU Qwen2.5_1.5:", critical_reasoning_data['qwen2_5_1_5b_bleu'].mean())
print("Mean BLEU Qwen2.5_14B:", critical_reasoning_data['qwen2_5_14b_bleu'].mean())

Mean BLEU DeepSeek_1.5: 0.018245463672188666
Mean BLEU DeepSeek_14B: 0.027657754041029277
Mean BLEU Qwen2.5_1.5: 0.05797982799515342
Mean BLEU Qwen2.5_14B: 0.07289545687764566


In [27]:
print("Mean ROUGE-L DeepSeek_1.5:", critical_reasoning_data['deepseek_r1_1_5b_rouge_l'].mean())
print("Mean ROUGE-L DeepSeek_14B:", critical_reasoning_data['deepseek_r1_14b_rouge_l'].mean())
print("Mean ROUGE-L Qwen2.5_1.5:", critical_reasoning_data['qwen2_5_1_5b_rouge_l'].mean())
print("Mean ROUGE-L Qwen2.5_14B:", critical_reasoning_data['qwen2_5_14b_rouge_l'].mean())

Mean ROUGE-L DeepSeek_1.5: 0.18589907220833218
Mean ROUGE-L DeepSeek_14B: 0.22016387565036785
Mean ROUGE-L Qwen2.5_1.5: 0.23310372304902421
Mean ROUGE-L Qwen2.5_14B: 0.2507744361820665


In [28]:
print("Mean METEOR DeepSeek_1.5:", critical_reasoning_data['deepseek_r1_1_5b_meteor'].mean())
print("Mean METEOR DeepSeek_14B:", critical_reasoning_data['deepseek_r1_14b_meteor'].mean())
print("Mean METEOR Qwen2.5_1.5:", critical_reasoning_data['qwen2_5_1_5b_meteor'].mean())
print("Mean METEOR Qwen2.5_14B:", critical_reasoning_data['qwen2_5_14b_meteor'].mean())

Mean METEOR DeepSeek_1.5: 0.14331880418782894
Mean METEOR DeepSeek_14B: 0.16255164560919616
Mean METEOR Qwen2.5_1.5: 0.2579807949967585
Mean METEOR Qwen2.5_14B: 0.3112524340289409


In [29]:
metrics = {
    "Cosine Similarity": [
        deepseek_r1_1_5b_similarity.mean(),
        deepseek_r1_14b_similarity.mean(),
        qwen2_5_1_5b_similarity.mean(),
        qwen2_5_14b_similarity.mean()
    ],
    "N-gram Overlap": [
        critical_reasoning_data['deepseek_r1_1_5b_ngram_overlap'].mean(),
        critical_reasoning_data['deepseek_r1_14b_ngram_overlap'].mean(),
        critical_reasoning_data['qwen2_5_1_5b_ngram_overlap'].mean(),
        critical_reasoning_data['qwen2_5_14b_ngram_overlap'].mean()
    ],
    "BLEU": [
        critical_reasoning_data['deepseek_r1_1_5b_bleu'].mean(),
        critical_reasoning_data['deepseek_r1_14b_bleu'].mean(),
        critical_reasoning_data['qwen2_5_1_5b_bleu'].mean(),
        critical_reasoning_data['qwen2_5_14b_bleu'].mean()
    ],
    "ROUGE-L": [
        critical_reasoning_data['deepseek_r1_1_5b_rouge_l'].mean(),
        critical_reasoning_data['deepseek_r1_14b_rouge_l'].mean(),
        critical_reasoning_data['qwen2_5_1_5b_rouge_l'].mean(),
        critical_reasoning_data['qwen2_5_14b_rouge_l'].mean()
    ],
    "METEOR": [
        critical_reasoning_data['deepseek_r1_1_5b_meteor'].mean(),
        critical_reasoning_data['deepseek_r1_14b_meteor'].mean(),
        critical_reasoning_data['qwen2_5_1_5b_meteor'].mean(),
        critical_reasoning_data['qwen2_5_14b_meteor'].mean()
    ]
}

models = ["DeepSeek R1 1.5B", "DeepSeek R1 14B", "Qwen2.5 1.5B", "Qwen2.5 14B"]
metrics_df = pd.DataFrame(metrics, index=models)
print(metrics_df)

                  Cosine Similarity  N-gram Overlap      BLEU   ROUGE-L  \
DeepSeek R1 1.5B           0.331343        0.007731  0.018245  0.185899   
DeepSeek R1 14B            0.388703        0.011949  0.027658  0.220164   
Qwen2.5 1.5B               0.423286        0.018782  0.057980  0.233104   
Qwen2.5 14B                0.453259        0.023330  0.072895  0.250774   

                    METEOR  
DeepSeek R1 1.5B  0.143319  
DeepSeek R1 14B   0.162552  
Qwen2.5 1.5B      0.257981  
Qwen2.5 14B       0.311252  
